In [ ]:
# !pip install pandas
# !pip install numpy 
# !pip install scikit-learn


In [1]:
import pandas as pd
import numpy as np
import tqdm

In [2]:
#import dataset
data = pd.read_csv('language_dataset.csv')

### Minor EDA   

In [3]:
#check for the shape of dataset and missing values
data.shape
data.isnull().sum()

Unnamed: 0    0
Text          0
language      0
dtype: int64

In [4]:
#drop unnecessary column
data.drop(columns='Unnamed: 0', inplace=True)
data.columns

Index(['Text', 'language'], dtype='object')

In [5]:
#check for number of observations per language
data['language'].value_counts()

language
Estonian      1000
Swedish       1000
English       1000
Russian       1000
Romanian      1000
Persian       1000
Pushto        1000
Spanish       1000
Hindi         1000
Korean        1000
Chinese       1000
French        1000
Portugese     1000
Indonesian    1000
Urdu          1000
Latin         1000
Turkish       1000
Japanese      1000
Dutch         1000
Tamil         1000
Thai          1000
Arabic        1000
Name: count, dtype: int64

#### Feature Extraction

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


In [7]:
#separate X and Y features
x_data = data[['Text']]
y_data = data[['language']]

In [8]:
#extract X features and check new shape
vectorizer = CountVectorizer()
x_data_vectorized = vectorizer.fit_transform(x_data['Text'])
x_data_vectorized.shape

(22000, 277720)

In [9]:
#split X and Y data with balanced number of observations in target variable
x_train, x_test, y_train, y_test = train_test_split(x_data_vectorized, y_data,
                                                     test_size= 0.3, 
                                                     random_state=34, 
                                                     stratify = y_data
                                                     )

#### Model Training

In [10]:
#import all modules
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,f1_score
from tqdm import tqdm

In [ ]:
#hyperparameter tuning for RF classifier
param_grid = {'max_depth': [5, 8, None],
              'max_features' : [5, 7, "auto"],
              'n_estimators': [50, 100, 150],
              'min_samples_split': [2, 8]
              }

new_rf = RandomForestClassifier(random_state= 12)
cv = GridSearchCV(estimator=new_rf, 
                  param_grid= param_grid,
                    cv= 5)
cv.fit(x_train, y_train['language'])
cv.best_estimator_

In [12]:
#specify model dictionary
models = {
     'Logistic Regression' : LogisticRegression(random_state=45),
     'Multinomial NB' : MultinomialNB(),
     'SVC' : SVC(random_state=21),
     'RF Classifier' : RandomForestClassifier(max_features=5,
                                              min_samples_split=8,
                                              n_estimators=150,
                                              random_state=12)
}


In [13]:
for i in tqdm(range(len(list(models)))) :
    #fit model
    model_object = list(models.values())[i]
    model = model_object.fit(x_train, y_train['language'])
    

    #predict and collect metrics
    y_predicted = model.predict(x_test)
    model_accuracy = accuracy_score(y_test, y_predicted)
    model_precision = precision_score(y_test, y_predicted, average= "weighted")
    model_recall = recall_score(y_test, y_predicted, average= "weighted")
    model_f1_score = f1_score(y_test, y_predicted, average= "weighted" )
    
    #print metrics
    print(list(models.keys())[i])
    print(f"Accuracy : {model_accuracy}")
    print (f"Precision: {model_precision}")
    print(f"Recall : {model_recall}")
    print(f"F1 score : {model_f1_score}" )
    print('=' *35)
    print('\n')

 25%|██▌       | 1/4 [00:33<01:40, 33.65s/it]

Logistic Regression
Accuracy : 0.9466666666666667
Precision: 0.9597950007946767
Recall : 0.9466666666666667
F1 score : 0.9490172242528225




 50%|█████     | 2/4 [00:34<00:28, 14.16s/it]

Multinomial NB
Accuracy : 0.9554545454545454
Precision: 0.9643046553216775
Recall : 0.9554545454545454
F1 score : 0.9552240505289885




 75%|███████▌  | 3/4 [01:37<00:36, 36.83s/it]

SVC
Accuracy : 0.894090909090909
Precision: 0.9196356565331157
Recall : 0.894090909090909
F1 score : 0.9003272492240816




100%|██████████| 4/4 [05:48<00:00, 87.11s/it] 

RF Classifier
Accuracy : 0.9409090909090909
Precision: 0.9663120721574046
Recall : 0.9409090909090909
F1 score : 0.932842299313773




### Fit final model

In [11]:
MNB = MultinomialNB()
MNB_model = MNB.fit(x_train, y_train['language'])

### Testing model on new dataset

In [77]:
#function to predict a dataframe of text
def predict_dataset(dataset):
 data_detected = False
 try: 
  #try to read as csv
  new_test_data = pd.read_csv(dataset + '.csv', delimiter= ",")
  data_detected = True
 except FileNotFoundError:
   try:
   #try to read as txt
     new_test_data = pd.read_csv(dataset + '.txt', delimiter= ",")
     data_detected = True
   
   finally:
     if data_detected == True:
      new_data_x = new_test_data['text']
      new_data_y = new_test_data['language']
      new_data_vectorized = vectorizer.transform(new_data_x)
      new_predictions_NB = MNB_model.predict(new_data_vectorized)
      accuracy = accuracy_score(new_data_y, new_predictions_NB)
      print(accuracy)
      

In [78]:

predict_dataset('multilingual-100')


0.8484848484848485


In [17]:
#function to predict a single text
def predict_text(text):
    text = vectorizer.transform([text])
    prediction = MNB_model.predict(text)
    print(prediction)

In [84]:
text = "La musique adoucit les mœurs et inspire l'âme"
predict_text(text)

['French']
